In [1]:
import json
import numpy as np

In [ ]:
def calculate_movement(sequence): # 연속된 프레임간의 평균 변화량 계산   
    diff = np.diff(sequence, axis=0)
    return np.mean(np.abs(diff))

def process_json_file(input_file, output_file, window_size=30, low_threshold=0.01, high_threshold=0.05):
    with open(input_file, 'r') as f:
        data = json.load(f)
    
    pose_data = data['pose_data']
    fall_start_frame = data.get('fall_start_frame', -1)
    fall_end_frame = data.get('fall_end_frame', -1)
    
    new_pose_data = {}
    landmark_history = []
    
    for frame, landmarks in pose_data.items():
        frame = int(frame)
        frame_landmarks = []
        for landmark in [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]:
            frame_landmarks.extend([
                landmarks[f'landmark_{landmark}']['x'],
                landmarks[f'landmark_{landmark}']['y']
            ])
        
        landmark_history.append(frame_landmarks)
        
        if len(landmark_history) > window_size:
            landmark_history.pop(0)
        
        if len(landmark_history) == window_size:
            movement = calculate_movement(np.array(landmark_history))
            
            # 클래스 분류
            if fall_start_frame <= frame <= fall_end_frame:
                class_label = "Fall Risk"
            elif movement > high_threshold:
                class_label = "Complete Fall"
            elif movement > low_threshold:
                class_label = "Fall Risk"
            else:
                class_label = "Non-Fall"
        else:
            movement = 0
            class_label = "Non-Fall"  # 초기 프레임들
        
        new_pose_data[frame] = {
            "landmarks": landmarks,
            "class": class_label,
            "movement": float(movement)
        }
    
    # 새로운 JSON 파일 생성
    new_data = {
        "pose_data": new_pose_data,
        "fall_start_frame": fall_start_frame,
        "fall_end_frame": fall_end_frame
    }
    
    with open(output_file, 'w') as f:
        json.dump(new_data, f, indent=4)

# 사용 예시
input_file = 'path/to/input.json'
output_file = 'path/to/output.json'
process_json_file(input_file, output_file)